In [1]:
import logging
import sys

import joblib
import lightgbm as lgb
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv("telco/telco-churn.csv")

In [3]:
df.sample(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5466,0221-NAUXK,Male,0,No,Yes,10,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Bank transfer (automatic),19.95,219.5,No
1850,3537-RYBHH,Female,1,Yes,No,47,Yes,Yes,DSL,Yes,...,Yes,No,No,No,Month-to-month,Yes,Credit card (automatic),67.45,3252,No
402,0979-PHULV,Male,0,Yes,Yes,69,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Credit card (automatic),99.45,7007.6,Yes
6241,8878-HMWBV,Male,0,No,No,3,Yes,No,DSL,No,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,49.15,169.05,Yes
3311,6137-MFAJN,Female,0,No,No,48,No,No phone service,DSL,Yes,...,No,No,Yes,No,Month-to-month,Yes,Electronic check,44.80,2104.55,No
1480,8898-KASCD,Male,0,No,No,39,No,No phone service,DSL,No,...,Yes,Yes,No,No,One year,No,Mailed check,35.55,1309.15,No
588,0742-LAFQK,Male,0,Yes,No,72,Yes,Yes,Fiber optic,Yes,...,Yes,No,No,Yes,Two year,Yes,Electronic check,99.15,7422.1,No
645,1626-ERCMM,Male,1,Yes,No,20,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.55,1899.65,Yes
423,3935-TBRZZ,Male,0,Yes,Yes,44,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,25.70,1110.5,No
5529,5919-VCZYM,Male,0,No,No,2,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.05,42.7,No


In [4]:
cat_features = list(df.loc[:, df.dtypes == 'object'].columns.values)
for feature in cat_features:
    df[feature] = pd.Series(df[feature], dtype="category")

In [5]:
X = df.drop(columns=["customerID", "Churn"], axis=1)
y = df["Churn"]

## Run A Study

In [6]:
def objective(optimize_boosting_type=True):
    def _objective(trial):
        if optimize_boosting_type:
            boosting_type = trial.suggest_categorical("boosting_type", ["dart", "gbdt"])
        else:
            boosting_type = "gbdt"
        lambda_l1 = trial.suggest_float(
            'lambda_l1', 1e-8, 10.0, log=True),
        lambda_l2 = trial.suggest_float(
            'lambda_l2', 1e-8, 10.0, log=True),
        num_leaves = trial.suggest_int(
            'num_leaves', 2, 256),
        feature_fraction = trial.suggest_float(
            'feature_fraction', 0.4, 1.0),
        bagging_fraction = trial.suggest_float(
            'bagging_fraction', 0.4, 1.0),
        bagging_freq = trial.suggest_int(
            'bagging_freq', 1, 7),
        min_child_samples = trial.suggest_int(
            'min_child_samples', 5, 100),
        learning_rate = trial.suggest_float(
            "learning_rate", 0.0001, 0.5, log=True),
        max_bin = trial.suggest_int(
            "max_bin", 128, 512, 32)
        n_estimators = trial.suggest_int(
            "n_estimators", 40, 400, 20)

        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "binary")

        model = lgb.LGBMClassifier(
            force_row_wise=True,
            boosting_type=boosting_type,
            n_estimators=n_estimators,
            lambda_l1=lambda_l1,
            lambda_l2=lambda_l2,
            num_leaves=num_leaves,
            feature_fraction=feature_fraction,
            bagging_fraction=bagging_fraction,
            bagging_freq=bagging_freq,
            min_child_samples=min_child_samples,
            learning_rate=learning_rate,
            max_bin=max_bin,
            callbacks=[pruning_callback],
            verbose=-1)
        scores = cross_val_score(model, X, y, scoring="f1_macro")
        return scores.mean()

    return _objective

In [7]:
sampler = optuna.samplers.TPESampler()
pruner = optuna.pruners.HyperbandPruner(
    min_resource=10, max_resource=400, reduction_factor=3)

study = optuna.create_study(
    direction='maximize', sampler=sampler,
    pruner=pruner
)
study.optimize(objective(), n_trials=100, gc_after_trial=True, n_jobs=-1)

[I 2023-04-27 10:28:12,658] A new study created in memory with name: no-name-4d5b595b-6933-4f5e-a075-54a3a8e6f4ba
[I 2023-04-27 10:28:17,653] Trial 6 finished with value: 0.6648478078588671 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.8685314248567396e-06, 'lambda_l2': 2.9848294452493614e-08, 'num_leaves': 19, 'feature_fraction': 0.9460072127370179, 'bagging_fraction': 0.6961435923195338, 'bagging_freq': 2, 'min_child_samples': 59, 'learning_rate': 0.028406303087725134, 'max_bin': 416, 'n_estimators': 40}. Best is trial 6 with value: 0.6648478078588671.
[I 2023-04-27 10:28:18,608] Trial 0 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.772915202694721e-07, 'lambda_l2': 1.820053164125362e-07, 'num_leaves': 25, 'feature_fraction': 0.8999729196142454, 'bagging_fraction': 0.8211338945242368, 'bagging_freq': 2, 'min_child_samples': 85, 'learning_rate': 0.00018924512031810915, 'max_bin': 352, 'n_estimators': 40}. Best is trial 6 wi

In [8]:
print(study.best_trial)

FrozenTrial(number=91, state=TrialState.COMPLETE, values=[0.7307756224362125], datetime_start=datetime.datetime(2023, 4, 27, 10, 39, 52, 816540), datetime_complete=datetime.datetime(2023, 4, 27, 10, 40, 33, 685912), params={'boosting_type': 'dart', 'lambda_l1': 0.0001299086075040824, 'lambda_l2': 0.002180292030688179, 'num_leaves': 6, 'feature_fraction': 0.581426005452652, 'bagging_fraction': 0.9302404000281413, 'bagging_freq': 5, 'min_child_samples': 54, 'learning_rate': 0.07460637901753928, 'max_bin': 224, 'n_estimators': 380}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatDis

## Understanding Parameters

In [27]:
fig = optuna.visualization.plot_parallel_coordinate(study, params=["boosting_type", "num_leaves", "learning_rate", "n_estimators"])
fig.write_image(file="figures/ch5_parallel_coordinates.png", format="png", scale=6)
fig.show()

In [17]:
fig = optuna.visualization.plot_param_importances(study)
fig.write_image(file="figures/ch5_importances.png", format="png", scale=6)
fig.show()

## Save and Resume a Study

In [19]:
joblib.dump(study, "lgbm-optuna-study.pkl")

['lgbm-optuna-study.pkl']

In [26]:
study = joblib.load("lgbm-optuna-study.pkl")
study.optimize(objective(), n_trials=20, gc_after_trial=True, n_jobs=-1)

In [13]:
print(study.best_trial)

FrozenTrial(number=113, state=TrialState.COMPLETE, values=[0.7325947131542604], datetime_start=datetime.datetime(2023, 4, 26, 21, 51, 41, 700790), datetime_complete=datetime.datetime(2023, 4, 26, 21, 51, 58, 269510), params={'boosting_type': 'gbdt', 'lambda_l1': 8.761930063313846e-06, 'lambda_l2': 3.674098080906251e-06, 'num_leaves': 2, 'feature_fraction': 0.8013834287179691, 'bagging_fraction': 0.8930729866672722, 'bagging_freq': 1, 'min_child_samples': 68, 'learning_rate': 0.14747560109267427, 'max_bin': 416, 'n_estimators': 140}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': Float

## Save and Resume on a remote DB

In [21]:
sampler = optuna.samplers.TPESampler()
pruner = optuna.pruners.HyperbandPruner(min_resource=10, max_resource=400, reduction_factor=3)

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "lgbm-tpe-rdb-study"
storage_name = f"sqlite:///{study_name}.db"
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    load_if_exists=False,
    sampler=sampler,
    pruner=pruner)
study.optimize(objective(False), n_trials=100, gc_after_trial=True, n_jobs=-1)

[I 2023-04-26 21:55:54,747] A new study created in RDB with name: lgbm-cma-es-rdb-study


A new study created in RDB with name: lgbm-cma-es-rdb-study


[I 2023-04-26 21:56:33,779] Trial 3 finished with value: 0.7132657507447453 and parameters: {'lambda_l1': 2.2532403757797748e-05, 'lambda_l2': 0.05662631493366982, 'num_leaves': 91, 'feature_fraction': 0.8673704694022113, 'bagging_fraction': 0.7408227257890989, 'bagging_freq': 3, 'min_child_samples': 69, 'learning_rate': 0.20682759099882317, 'max_bin': 512, 'n_estimators': 60}. Best is trial 3 with value: 0.7132657507447453.


Trial 3 finished with value: 0.7132657507447453 and parameters: {'lambda_l1': 2.2532403757797748e-05, 'lambda_l2': 0.05662631493366982, 'num_leaves': 91, 'feature_fraction': 0.8673704694022113, 'bagging_fraction': 0.7408227257890989, 'bagging_freq': 3, 'min_child_samples': 69, 'learning_rate': 0.20682759099882317, 'max_bin': 512, 'n_estimators': 60}. Best is trial 3 with value: 0.7132657507447453.


[I 2023-04-26 21:56:34,880] Trial 4 finished with value: 0.7226552433871103 and parameters: {'lambda_l1': 0.006182106804178753, 'lambda_l2': 6.160625321836477, 'num_leaves': 77, 'feature_fraction': 0.941323475749757, 'bagging_fraction': 0.4142484304524909, 'bagging_freq': 6, 'min_child_samples': 94, 'learning_rate': 0.017326662146954618, 'max_bin': 512, 'n_estimators': 180}. Best is trial 3 with value: 0.7132657507447453.


...

[I 2023-04-26 22:02:59,748] Trial 50 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0012739682245968972, 'lambda_l2': 9.842818164648099e-05, 'num_leaves': 147, 'feature_fraction': 0.5631666822474554, 'bagging_fraction': 0.7274109201998455, 'bagging_freq': 5, 'min_child_samples': 53, 'learning_rate': 0.0022965647473676305, 'max_bin': 416, 'n_estimators': 180}. Best is trial 5 with value: 0.4235082247366586.


Trial 50 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0012739682245968972, 'lambda_l2': 9.842818164648099e-05, 'num_leaves': 147, 'feature_fraction': 0.5631666822474554, 'bagging_fraction': 0.7274109201998455, 'bagging_freq': 5, 'min_child_samples': 53, 'learning_rate': 0.0022965647473676305, 'max_bin': 416, 'n_estimators': 180}. Best is trial 5 with value: 0.4235082247366586.


[I 2023-04-26 22:03:00,457] Trial 49 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0050038474922444315, 'lambda_l2': 5.162993438119722e-06, 'num_leaves': 103, 'feature_fraction': 0.5098782589132528, 'bagging_fraction': 0.677029585535416, 'bagging_freq': 3, 'min_child_samples': 52, 'learning_rate': 0.00020630631963538095, 'max_bin': 448, 'n_estimators': 220}. Best is trial 5 with value: 0.4235082247366586.


Trial 49 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0050038474922444315, 'lambda_l2': 5.162993438119722e-06, 'num_leaves': 103, 'feature_fraction': 0.5098782589132528, 'bagging_fraction': 0.677029585535416, 'bagging_freq': 3, 'min_child_samples': 52, 'learning_rate': 0.00020630631963538095, 'max_bin': 448, 'n_estimators': 220}. Best is trial 5 with value: 0.4235082247366586.


RuntimeError: dictionary changed size during iteration

In [12]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective(), n_trials=20, gc_after_trial=True, n_jobs=-1)

[I 2023-04-21 08:25:32,162] Using an existing study with name 'lgbm-optuna-rdb-study' instead of creating a new one.


Using an existing study with name 'lgbm-optuna-rdb-study' instead of creating a new one.


[I 2023-04-21 08:26:17,345] Trial 16 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.0179602799796386e-08, 'lambda_l2': 9.349435489679461, 'num_leaves': 247, 'feature_fraction': 0.7460125770492829, 'bagging_fraction': 0.8227537764241121, 'bagging_freq': 7, 'min_child_samples': 98, 'learning_rate': 0.00020887902150769306, 'max_bin': 512, 'n_estimators': 140}. Best is trial 2 with value: 0.4235082247366586.


Trial 16 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.0179602799796386e-08, 'lambda_l2': 9.349435489679461, 'num_leaves': 247, 'feature_fraction': 0.7460125770492829, 'bagging_fraction': 0.8227537764241121, 'bagging_freq': 7, 'min_child_samples': 98, 'learning_rate': 0.00020887902150769306, 'max_bin': 512, 'n_estimators': 140}. Best is trial 2 with value: 0.4235082247366586.


[I 2023-04-21 08:26:17,706] Trial 10 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.1909111890008977e-08, 'lambda_l2': 3.916792214730818, 'num_leaves': 252, 'feature_fraction': 0.7842462593069699, 'bagging_fraction': 0.8275752521380275, 'bagging_freq': 7, 'min_child_samples': 100, 'learning_rate': 0.00011984328761926979, 'max_bin': 512, 'n_estimators': 140}. Best is trial 2 with value: 0.4235082247366586.


...

Trial 14 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.4408133646747365e-08, 'lambda_l2': 2.837493258342436, 'num_leaves': 245, 'feature_fraction': 0.7637633482746569, 'bagging_fraction': 0.8296627735915492, 'bagging_freq': 7, 'min_child_samples': 37, 'learning_rate': 0.00013694284525827822, 'max_bin': 512, 'n_estimators': 160}. Best is trial 2 with value: 0.4235082247366586.


[I 2023-04-21 08:26:34,243] Trial 19 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.3957343825730509e-06, 'lambda_l2': 1.2256972812369361e-08, 'num_leaves': 5, 'feature_fraction': 0.8427033256168555, 'bagging_fraction': 0.5366577275595632, 'bagging_freq': 1, 'min_child_samples': 36, 'learning_rate': 0.0007048212563581564, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


Trial 19 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.3957343825730509e-06, 'lambda_l2': 1.2256972812369361e-08, 'num_leaves': 5, 'feature_fraction': 0.8427033256168555, 'bagging_fraction': 0.5366577275595632, 'bagging_freq': 1, 'min_child_samples': 36, 'learning_rate': 0.0007048212563581564, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


[I 2023-04-21 08:26:34,762] Trial 18 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.2876121235965678e-06, 'lambda_l2': 1.542472294392896e-07, 'num_leaves': 4, 'feature_fraction': 0.8472764387650585, 'bagging_fraction': 0.5063448986999544, 'bagging_freq': 1, 'min_child_samples': 34, 'learning_rate': 0.0005789551103378422, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


Trial 18 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.2876121235965678e-06, 'lambda_l2': 1.542472294392896e-07, 'num_leaves': 4, 'feature_fraction': 0.8472764387650585, 'bagging_fraction': 0.5063448986999544, 'bagging_freq': 1, 'min_child_samples': 34, 'learning_rate': 0.0005789551103378422, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


In [23]:
print(study.best_trial)

FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.4235082247366586], datetime_start=datetime.datetime(2023, 4, 26, 21, 55, 55, 15345), datetime_complete=datetime.datetime(2023, 4, 26, 21, 56, 38, 244296), params={'bagging_fraction': 0.9277029430467971, 'bagging_freq': 3, 'feature_fraction': 0.6565211338929926, 'lambda_l1': 4.496791174062579, 'lambda_l2': 0.000612013760106219, 'learning_rate': 0.00011283703507767392, 'max_bin': 288, 'min_child_samples': 97, 'n_estimators': 180, 'num_leaves': 100}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_freq': IntDistribution(high=7, log=False, low=1, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'learning_rate': FloatDistribution(high=0

## Multi-objective optimization

In [22]:
def moo_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.5, log=True),

    model = lgb.LGBMClassifier(
        force_row_wise=True,
        boosting_type='gbdt',
        n_estimators=200,
        lambda_l1=3.298803078077973e-07,
        lambda_l2=8.938532783741386e-07,
        num_leaves=6,
        feature_fraction=0.5133218336120866,
        bagging_fraction=0.9660809666082303,
        bagging_freq=7,
        min_child_samples=91,
        learning_rate=learning_rate,
        max_bin=320,
        verbose=-1,
    )
    scores = cross_val_score(model, X, y, scoring="f1_macro")
    return learning_rate[0], scores.mean()

In [23]:
study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(moo_objective, n_trials=100)

[I 2023-04-27 10:47:35,501] A new study created in memory with name: no-name-eb122531-f2ee-4cb5-92d1-8efba6a28eef
[I 2023-04-27 10:47:36,221] Trial 0 finished with values: [0.018276625572235056, 0.7229728382357858] and parameters: {'learning_rate': 0.018276625572235056}. 
[I 2023-04-27 10:47:36,922] Trial 1 finished with values: [0.012281350875017864, 0.713129498994182] and parameters: {'learning_rate': 0.012281350875017864}. 
[I 2023-04-27 10:47:37,601] Trial 2 finished with values: [0.3179931543219117, 0.7181352358216049] and parameters: {'learning_rate': 0.3179931543219117}. 
[I 2023-04-27 10:47:38,317] Trial 3 finished with values: [0.3317048028124761, 0.7182462105095208] and parameters: {'learning_rate': 0.3317048028124761}. 
[I 2023-04-27 10:47:39,042] Trial 4 finished with values: [0.010097499163932589, 0.689595604293249] and parameters: {'learning_rate': 0.010097499163932589}. 
[I 2023-04-27 10:47:39,716] Trial 5 finished with values: [0.0010284892903758126, 0.4235082247366586]

In [24]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["learning_rate", "f1"])
fig.write_image(file="figures/ch5_pareto.png", format="png", scale=6)
fig.show()